# Business Case: Audiobooks

#### Agustín Giménez, May 2022

In [1]:
# import libraries
import numpy as np
from sklearn import preprocessing
import tensorflow as tf

### Preprocessing

In [2]:
# extract data from csv file
raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter=',')

# remove first column (IDs) and last column (targets)
unscaled_inputs_all = raw_csv_data[:,1:-1]

targets_all = raw_csv_data[:,-1]

#### Shuffle the data

In [3]:
# we do not want batches to have data from same/close dates
shuffled_indices = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indices)

unscaled_inputs_all = unscaled_inputs_all[shuffled_indices]
targets_all = targets_all[shuffled_indices]

#### Balance the dataset

In [4]:
# count the number of targets that are one's
num_one_targets = int(np.sum(targets_all))

zero_targets_counter = 0

indices_to_remove = []

# iterate through all targets to count the number of 0's to remove
for i in range(targets_all.shape[0]):
    if targets_all[i]==0:
        zero_targets_counter +=1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

#### Standardize the inputs

In [5]:
# use sklearn
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

#### Shuffle the data

In [6]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

#### Split the dataset into train, validation and test

In [7]:
# Count the total number of samples
samples_count = shuffled_inputs.shape[0]

# Count the samples in each subset, assuming we want 80-10-10 distribution of training, validation, and test.
# Naturally, the numbers are integers.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

# The 'test' dataset contains all remaining data.
test_samples_count = samples_count - train_samples_count - validation_samples_count

# Create variables that record the inputs and targets for training
# In our shuffled dataset, they are the first "train_samples_count" observations
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Create variables that record the inputs and targets for validation.
# They are the next "validation_samples_count" observations, folllowing the "train_samples_count" we already assigned
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# Create variables that record the inputs and targets for test.
# They are everything that is remaining.
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

In [8]:
print(np.sum(train_targets),train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets),validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets),test_samples_count, np.sum(test_targets) / test_samples_count)

1819.0 3579 0.5082425258452081
204.0 447 0.4563758389261745
214.0 448 0.47767857142857145


#### Save the three datasets  in *.npz format

In [9]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

## Create the Machine Learning Algorithm

### Data

In [10]:
# NOTE: Our data is in array form
# load the training data
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(float)
train_targets = npz['targets'].astype(int)

# load the validation data
npz = np.load('Audiobooks_data_validation.npz')
validation_inputs , validation_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

# load the target data
npz = np.load('Audiobooks_data_validation.npz')
test_inputs , test_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

### Model

Outline, optimizers, loss, early stopping and training

In [11]:
input_size = 10
output_size = 2 # our targets are 0's and 1's

hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    tf.keras.layers.Dense(output_size, activation='softmax'), # output layer
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size =100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets,
          batch_size=batch_size,
          epochs = max_epochs,
          callbacks=[early_stopping],
          validation_data = (validation_inputs, validation_targets),
          verbose=2
         )

Epoch 1/100
36/36 - 1s - loss: 0.6067 - accuracy: 0.6706 - val_loss: 0.5343 - val_accuracy: 0.7562 - 1s/epoch - 28ms/step
Epoch 2/100
36/36 - 0s - loss: 0.5031 - accuracy: 0.7569 - val_loss: 0.4681 - val_accuracy: 0.7696 - 275ms/epoch - 8ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4519 - accuracy: 0.7731 - val_loss: 0.4296 - val_accuracy: 0.7852 - 288ms/epoch - 8ms/step
Epoch 4/100
36/36 - 0s - loss: 0.4245 - accuracy: 0.7832 - val_loss: 0.4085 - val_accuracy: 0.8009 - 261ms/epoch - 7ms/step
Epoch 5/100
36/36 - 0s - loss: 0.4078 - accuracy: 0.7860 - val_loss: 0.3939 - val_accuracy: 0.7875 - 213ms/epoch - 6ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3944 - accuracy: 0.7890 - val_loss: 0.3869 - val_accuracy: 0.7964 - 83ms/epoch - 2ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3859 - accuracy: 0.7955 - val_loss: 0.3809 - val_accuracy: 0.8076 - 81ms/epoch - 2ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3808 - accuracy: 0.7955 - val_loss: 0.3707 - val_accuracy: 0.8277 - 85ms/epoch - 2ms/step
Epoch 9/100
3

### Testing

In [12]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3466 - accuracy: 0.8166
